In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import numpy as np

# โหลดตัวแปรจาก .env
load_dotenv()

# ดึงค่าจาก environment
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

# สร้าง engine สำหรับเชื่อมต่อฐานข้อมูล
engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

# SQL query
query = """
SELECT quo_num,company, company_prb,assured_insurance_capital1,is_addon,type,repair_type
FROM fin_system_select_plan WHERE datestart >= '2025-06-01' AND datestart < '2025-07-01' and type_insure = 'ประกันรถ'
 
"""


# โหลดข้อมูลจากฐานข้อมูล
df = pd.read_sql(query, engine)

# แปลงให้ pandas เข้าใจได้แน่นอน
# df['user_registered'] = pd.to_datetime(df['user_registered'].astype(str), errors='coerce')
df


In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv() 

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance_task'

engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

query = """
SELECT quo_num,responsibility1,responsibility2,responsibility3,responsibility4,damage1,damage2,damage3,damage4,protect1,protect2,protect3,protect4,
if(sendtype = 'ที่อยู่ใหม่',provincenew,province) as delivery_province,
show_ems_price,show_ems_type

FROM fin_order WHERE datekey >= '2025-06-01' AND datekey < '2025-07-01'

"""

df1 = pd.read_sql(query, engine)
df1


In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv() 

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

query = """
SELECT quo_num,date_warranty,date_exp

FROM fin_system_pay WHERE datestart >= '2025-06-01' AND datestart < '2025-07-01'

"""

df2 = pd.read_sql(query, engine)
df2


In [ ]:
df_merged = df.copy()

df_merged = pd.merge(df_merged, df1, on='quo_num', how='left')
df_merged = pd.merge(df_merged, df2, on='quo_num', how='left')


In [ ]:
def combine_columns(a, b):
    a_str = str(a).strip() if pd.notna(a) else ''
    b_str = str(b).strip() if pd.notna(b) else ''
    
    if a_str == '' and b_str == '':
        return ''
    elif a_str == '':
        return b_str
    elif b_str == '':
        return a_str
    elif a_str == b_str:
        return a_str
    else:
        return f"ชั้น{a_str} {b_str}"

df_merged['insurance_class'] = df_merged.apply(lambda row: combine_columns(row['type'], row['repair_type']), axis=1)


In [ ]:
df_merged

In [ ]:
for col in df_merged.columns:
    print(col)

In [ ]:
df_merged = df_merged.drop(columns=['type','repair_type'])
df_merged

In [ ]:
for col in df.columns:
    print(col)
##car_year

In [ ]:
rename_columns = {
    "quo_num": "quotation_num",
    "responsibility1": "human_coverage_person",
    "responsibility2": "human_coverage_atime",
    "responsibility3": "property_coverage",
    "responsibility4": "deductible",
    "damage1": "vehicle_damage",
    "damage2": "deductible_amount",
    "damage3": "vehicle_theft_fire",
    "damage4": "vehicle_flood_damage",
    "protect1": "personal_accident_driver",
    "protect2": "personal_accident_passengers",
    "protect3": "medical_coverage",
    "protect4": "driver_coverage",
    "company": "ins_company",
    "company_prb": "act_ins_company",
    "assured_insurance_capital1": "sum_insured",
    "date_warranty": "date_warranty",
    "date_exp": "date_expired",
    "insurance_class": "insurance_class",
    "show_ems_price": "ems_amount",
    "show_ems_type": "delivery_type",
    "is_addon": "income_comp_ins"
}

df = df_merged.rename(columns=rename_columns)
df

In [ ]:
df

In [ ]:
df['insurance_class'].unique()


In [ ]:
print(set(df_cleaned.columns))


In [ ]:
print(df_cleaned.columns.tolist())


In [ ]:
df = df.replace(r'^\s*$', pd.NA, regex=True)  
df = df[df.count(axis=1) > 1]
df

In [ ]:
import pandas as pd
import numpy as np

# แปลงช่องว่างทุกแบบเป็น NaN ชั่วคราว เพื่อการนับข้อมูล
df_temp = df.replace(r'^\s*$', np.nan, regex=True)

# เพิ่มคอลัมน์ช่วยนับจำนวนข้อมูล (non-null)
df['non_empty_count'] = df_temp.notnull().sum(axis=1)

# >>>> ส่วนที่แก้ไขตรงนี้ <<<<
# ตรวจสอบ agent_id ที่ไม่ว่าง (ไม่ใช่ NaN และไม่ใช่ช่องว่าง)
valid_agent_id_mask = df['quotation_num'].astype(str).str.strip().ne('') & df['quotation_num'].notna()

# แยกกลุ่มที่ agent_id ไม่ว่างและ agent_id ว่าง
df_with_id = df[valid_agent_id_mask]
df_without_id = df[~valid_agent_id_mask]

# คัดแถวที่ agent_id ซ้ำ โดยเก็บแถวที่มีข้อมูลมากที่สุด
df_with_id_cleaned = df_with_id.sort_values('non_empty_count', ascending=False).drop_duplicates(subset='quotation_num', keep='first')

# รวมกลับ
df_cleaned = pd.concat([df_with_id_cleaned, df_without_id], ignore_index=True)

# ลบคอลัมน์ช่วย
df_cleaned = df_cleaned.drop(columns=['non_empty_count'])
df_cleaned = df_cleaned.replace(
    to_replace=r'^\s*$|(?i:^none$)|^-$',  # << แก้ตรงนี้
    value=np.nan,
    regex=True
)


df_cleaned.columns = df_cleaned.columns.str.lower()
df_cleaned


In [ ]:
df_cleaned.replace(np.nan, "NaN").isin(["none", "-", "None"]).sum()
df_cleaned

In [ ]:
# df_cleaned.to_csv('dim_car1.csv', index=False, encoding='utf-8-sig')

In [ ]:
df_cleaned = df_cleaned.replace(r'^\.$', np.nan, regex=True)
df_cleaned

In [ ]:
df_cleaned['income_comp_ins'] = df_cleaned['income_comp_ins'].apply(lambda x: True if x == 1 else False if x == 0 else None)


In [ ]:
cols_to_clean = [
    'sum_insured', 'human_coverage_person', 'human_coverage_atime',
    'property_coverage', 'deductible', 'vehicle_damage', 'deductible_amount',
    'vehicle_theft_fire', 'vehicle_flood_damage', 'personal_accident_driver',
    'personal_accident_passengers', 'medical_coverage', 'driver_coverage',
    'ems_amount'
]
for col in cols_to_clean:
    df_cleaned[col] = (
        df_cleaned[col]
        .astype(str)
        .str.replace(',', '', regex=False)
        .str.strip()
        .apply(lambda x: float(x) if x not in ['', 'None', 'nan', 'NaN'] else None)
    )

In [ ]:
df_cleaned = df_cleaned.where(pd.notnull(df_cleaned), None)


In [ ]:
import numpy as np

df_cleaned['insurance_class'] = df_cleaned['insurance_class'].replace({
    'ซ่อมอู่': np.nan  # ใช้ NaN ซึ่งจะกลายเป็น NULL ตอนเขียนลง PostgreSQL
})


In [ ]:
df_cleaned['date_warranty'] = pd.to_datetime(df_cleaned['date_warranty'], errors='coerce')
df_cleaned['date_expired'] = pd.to_datetime(df_cleaned['date_expired'], errors='coerce')


In [ ]:
import numpy as np
import pandas as pd

int_columns = [
    "sum_insured", "human_coverage_person", "human_coverage_atime", "property_coverage",
    "deductible", "vehicle_damage", "deductible_amount", "vehicle_theft_fire",
    "vehicle_flood_damage", "personal_accident_driver", "personal_accident_passengers",
    "medical_coverage", "driver_coverage", "ems_amount"
]

for col in int_columns:
    if col in df_cleaned.columns:
        # แปลงเป็น numeric coercing error เป็น NaN
        df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
        # ถ้าอยากเก็บ NULL ในฐานข้อมูล → ใช้ dtype "Int64" ของ Pandas
        df_cleaned[col] = df_cleaned[col].astype("Int64")

# แปลง NaN เป็น None เพื่อเตรียม insert
df_cleaned = df_cleaned.where(pd.notnull(df_cleaned), None)
df_cleaned = df_cleaned.replace({np.nan: None})

print("✅ จัดการ column int เรียบร้อยแล้ว")


In [ ]:
df_cleaned

In [ ]:
import numpy as np
import pandas as pd
import os
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.dialects.postgresql import insert  # ✅ ใช้ insert แบบ Postgres
from dotenv import load_dotenv

load_dotenv()

host = os.getenv('DB_HOST_test')
user = os.getenv('DB_USER_test')
password = os.getenv('DB_PASSWORD_test')
port = os.getenv('DB_PORT_test')
database = 'fininsurance'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# ✅ ทำให้ NaN เป็น None
df_cleaned = df_cleaned.where(pd.notnull(df_cleaned), None)
df_cleaned = df_cleaned.replace({np.nan: None})

# ✅ กำหนด columns int ถ้ามี
int_columns = [
    "deductible", "human_coverage_person", "human_coverage_atime", "property_coverage",
    "personal_accident_driver", "personal_accident_passengers", "medical_coverage",
    "driver_coverage", "sum_insured", "ems_amount", "vehicle_damage", "deductible_amount",
    "vehicle_theft_fire", "vehicle_flood_damage"
]
for col in int_columns:
    if col in df_cleaned.columns:
        df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce').round()

df_cleaned = df_cleaned.where(pd.notnull(df_cleaned), None)

# ✅ Upsert function
def upsert_df_chunk_force(df: pd.DataFrame, table_name: str, key_column: str, chunk_size: int = 2000, schema: str = 'public'):
    metadata = MetaData(schema=schema)
    table = Table(table_name, metadata, autoload_with=engine)

    df = df[df[key_column].notna()]
    
    total_rows = len(df)
    total_chunks = (total_rows // chunk_size) + (1 if total_rows % chunk_size else 0)

    print(f"🎯 Row ทั้งหมด: {total_rows:,} (แบ่ง {total_chunks} ชุด, chunk size: {chunk_size})")

    for i in range(0, total_rows, chunk_size):
        df_chunk = df.iloc[i:i+chunk_size]
        rows = df_chunk.to_dict(orient='records')
        
        stmt = insert(table).values(rows)
        update_dict = {c.name: getattr(stmt.excluded, c.name) for c in table.columns if c.name != key_column}

        stmt = stmt.on_conflict_do_update(
            index_elements=[key_column],
            set_=update_dict
        )

        with engine.begin() as conn:
            result = conn.execute(stmt)
            affected_rows = result.rowcount

        print(f"   🚀 Chunk {i//chunk_size+1}/{total_chunks} ✔️ affected: {affected_rows:,}")

    print("✅🎯 Upsert ทุก row เสร็จสมบูรณ์!")

# ✅ เรียกใช้
upsert_df_chunk_force(df_cleaned, 'fact_insurance_motor', 'quotation_num')
